## 챌린지
---
(EN)
- Implement an LCEL chain with a memory that uses one of the memory classes we learned about.
- The chain should take the title of a movie and reply with three emojis that represent the movie. (i.e "Top Gun" -> "🛩️👨‍✈️🔥". "The Godfather" -> "👨‍👨‍👦🔫🍝 ").
- Provide examples to the chain using FewShotPromptTemplate or FewShotChatMessagePromptTemplate to make sure it always replies with three emojis.
- To check that the memory is working ask the chain about two movies and then in another cell ask the chain to tell you what is the movie you asked about first.

(KR)
- 앞서 배운 메모리 클래스 중 하나를 사용하는 메모리로 LCEL 체인을 구현합니다.
- 이 체인은 영화 제목을 가져와 영화를 나타내는 세 개의 이모티콘으로 응답해야 합니다. (예: "탑건" -> "🛩️👨‍✈️🔥". "대부" -> "👨‍👨‍👦🔫🍝").
- 항상 세 개의 이모티콘으로 답장하도록 FewShotPromptTemplate 또는 FewShotChatMessagePromptTemplate을 사용하여 체인에 예시를 제공하세요.
- 메모리가 작동하는지 확인하려면 체인에 두 개의 영화에 대해 질문한 다음 다른 셀에서 체인에 먼저 질문한 영화가 무엇인지 알려달라고 요청하세요.

In [2]:
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationBufferMemory

In [ ]:
llm = ChatOpenAI(temperature=0.1)

memory = ConversationBufferMemory()

# Few-shot examples
examples = [
    {"movie": "Titanic", "emoji": "🚢💑💔"},
    {"movie": "Jurassic Park", "emoji": "🦖🌿🚙"},
    {"movie": "Harry Potter", "emoji": "🧙‍♂️⚡📖"}
]

# Example prompt
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{movie}"),
    ("ai", "{emoji}")
])

# FewShotChatMessagePromptTemplate 적용
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

# FewShot 예제 부분을 문자열로 변환
formatted_few_shot = "\n".join(
    [f"Human: {ex['movie']}\nAI: {ex['emoji']}" for ex in examples]
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", f"""
    Here are some examples of movies and their corresponding emojis:\n\n{formatted_few_shot}\n\n. If the last human question is one of the movie titles, respond with exactly three emoticons. If the question is about remembering the movie you asked, respond with the movie you asked. Except for the examples above, you should respond based on your chat history.
    """),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}")
])

def load_memory(_):
    """메모리에서 저장된 대화를 불러와 LangChain 메시지 형식으로 변환"""
    history = memory.load_memory_variables({}).get("history", [])
    return history if isinstance(history, list) else []

In [45]:
chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm

def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context({"input": question}, {"output": result.content})
    print(result.content)

In [46]:
# 테스트 실행
invoke_chain("Inception")
invoke_chain("The Matrix")

🕰️🧠🌌
🕶️💊🤖


In [47]:
# 메모리 확인
print(memory.load_memory_variables({}))

{'history': 'Human: Inception\nAI: 🕰️🧠🌌\nHuman: The Matrix\nAI: 🕶️💊🤖'}


In [60]:
invoke_chain("What was the movie I asked about before 'The Matrix'?")

The movie you asked about before 'The Matrix' was 'Jurassic Park'. 🦖🌿🚙


## 정답 코드

In [4]:
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate

llm = ChatOpenAI(temperature=0)

memory = ConversationBufferMemory(
    llm=llm,
    return_messages=True,
)

examples = [
    {
        "question": "Spider Man",
        "answer": "🕷️🕸️🗽",
    },
    {
        "question": "Iron Man",
        "answer": "🦾🕶️🔥",
    },
    {
        "question": "Thor",
        "answer": "⚡️🔨🌩️",
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [("human", "{question}"), ("ai", "{answer}")]
)

fewshot_chat_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a movie expert. You know every movie. If a human tells you the title of the movie, you have to respond with 3 emoticons.",
        ),
        fewshot_chat_prompt,
        (
            "system",
            "The above examples should not be provided to the user. The user can only be provided with the conversation record below. Please provide the information to the user using the record below.",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm


def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context({"input": question}, {"output": result.content})
    print(result.content)

In [5]:
invoke_chain("Captain America")
invoke_chain("Mission Impossible")

🇺🇸⭐️🛡️
🕶️💣🔥


In [6]:
invoke_chain("What was the first movie I asked?")

You asked about "Captain America."
